In [1]:
from datetime import datetime
import pytz
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import sql
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
df = pd.read_csv(r"C:\Users\manue\Desktop\data_eng\data_eng\_02_yahoof\symbol_name\symbol_name.csv")
# lista de tickers
tickers = df['symbol'].tolist()

# Definir el rango de fechas
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=60)  # Cambié esto a 60 días

# Crear un DataFrame vacío para almacenar los datos
all_data = pd.DataFrame()

# Iterar sobre cada ticker y obtener los datos históricos
for ticker in tickers:
    try:
        data = yf.download(ticker, start=start_date, end=end_date, interval="5m")
        data['Symbol'] = ticker  # Agregar una columna para el símbolo del ticker
        all_data = pd.concat([all_data, data])  # Concatenar los datos al DataFrame principal
    except Exception as e:
        print(f"No se pudieron obtener los datos para el ticker {ticker}: {e}")

all_data['date_time'] = all_data.index
all_data = all_data.reset_index(drop=True)

#todas las columans en minisculas y los espacios unidos por underscore
all_data = all_data.rename(columns=lambda x: x.lower().replace(' ', '_'))
#Renombramos la columna por ticker
all_data = all_data.rename(columns = {'symbol' : 'ticker'})

#Cambia la zona horaria para que no hayan conflictos al subirlo a PSQL
all_data['date_time'] = pd.to_datetime(all_data['date_time']).dt.tz_localize(None)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [5]:
# Datos de conexión a la base de datos
dbname = 'yfinance'
user = 'postgres'
password = 'mjose2002'
host = 'localhost'  # O el host donde se encuentra tu base de datos
port = '5432'       # El puerto por defecto de PostgreSQL

# Conexión a la base de datos
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Conexión exitosa a la base de datos")
except psycopg2.Error as e:
    print("Error al conectarse a la base de datos:", e)
    exit()

# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

# Nombre de la tabla en la base de datos
table_name = 'stock_data'

# Crea la cadena de conexión
connection_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Crea el motor de conexión
engine = create_engine(connection_str)

# Cargar los datos del DataFrame en la tabla de la base de datos
all_data.to_sql(table_name, engine, if_exists='append', index=False)

# Confirmar la ejecución de la transacción
conn.commit()

# Cerrar el cursor y la conexión
cur.close()
conn.close()

Conexión exitosa a la base de datos
